In [2]:
'''
colab에서 실행시 아래 명령어 실행
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy
'''
from konlpy.tag import Mecab
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from gensim.models import word2vec
import re
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter

In [3]:
def get_korean_morphs(words) :
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    # print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.'))
    return mecab.morphs(words)
#########################
str = '영등포 맛집 알려줘'
get_korean_morphs(str)

['영등포', '맛집', '알려줘']

# Stopwords   / 불용어 제거하기 / 필요없는 단어 제거하기

In [7]:
def get_korean_clean_word(words, stopwords):
    nouns = []
    tagger = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    for post in words:
        for noun in tagger.nouns(post):
            if noun not in stopwords:
                nouns.append(noun)  # stopwords에 없을 때, 사전에다 넣어줘라!
    return nouns
######################
words = ['중국','한국', '중요', '이웃', '협력', '파트너', '불안', '층']
stopwords = ['중국','한국']
get_korean_clean_word(words,stopwords)

['중요', '이웃', '협력', '파트너', '불안', '층']

In [ ]:
def get_korean_pos_tag(words):
    words = []
    tagger = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    for post in words:
        words.extend(tagger.pos(post))
    return words
###################3


In [10]:
#ask!!!?????????????????????
def get_korean_noun_list(malist):
    word_dic = {}
    #malist = [('사랑', 'Noun'), ('이', '조사'), ('사랑', 'Noun')] # 얘가 dictionary 타입이여서 0번지, 1번지   형태로 되어 있다.
    for word in malist:
        if word[1] == "Noun":  # 명사 확인하기
            if not (word[0] in word_dic):
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1  # 카운트하기
    keys = sorted(word_dic.items(), key=lambda x: x[1], reverse=True)
    return keys

def get_korean_noun_list2(malist):
    word_dic = {}
    #malist = [('사랑', 'Noun'), ('이', '조사'), ('사랑', 'Noun')] # 얘가 dictionary 타입이여서 0번지, 1번지   형태로 되어 있다.
    for word in malist:
        if word[1] == "조사":  # 명사 확인하기
            if not (word[0] in word_dic):
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1  # 카운트하기
    keys = sorted(word_dic.items(), key=lambda x: x[1], reverse=True)
    return keys


malist = [('사랑', 'Noun'), ('이', '조사'), ('사랑', 'Noun'), ('이', 'Noun')]
print(get_korean_noun_list(malist))
print(get_korean_noun_list2(malist))


[('사랑', 2), ('이', 1)]
[('이', 1)]


In [ ]:
def sort_by_keys(dict):
    return sorted(value = dict.keys(), reverse=True)

def sort_by_values(dict):
    return sorted(value = dict, key=dict.get, reverse=True)

# 빈도수가 높은 단어

In [14]:
def get_most_common_words(word_list, num):
    # num_top_nouns = 20
    counter = Counter(word_list)
    top_words = dict(counter.most_common(num))
    return top_words

word_list = ['중국','한국','중국','한국','한국', '중요', '이웃', '협력', '파트너', '불안', '층']

print(get_most_common_words(word_list,1))
print(get_most_common_words(word_list,2))
print(get_most_common_words(word_list,3))

{'한국': 3}
{'한국': 3, '중국': 2}
{'한국': 3, '중국': 2, '중요': 1}


In [ ]:
def draw_word_cloud(word_list):
    wc = WordCloud(background_color="white", font_path='./font/NanumBarunGothic.ttf')
    wc.generate_from_frequencies(word_list)
    figure = plt.figure()
    figure.set_size_inches(10, 10)
    ax = figure.add_subplot(1, 1, 1)
    ax.axis("off")
    ax.imshow(wc)

def draw_word_cloud_with_mask(word_list, imagepath):
    mask = np.array(Image.open(imagepath))
    # 워드 클라우드 설정
    wc = WordCloud(background_color="white", mask=mask, contour_width=3,
                         font_path='../font/NanumBarunGothic.ttf')
    wc.generate_from_frequencies(word_list)
    # 이미지 표시
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    # 이미지 저장
    wc.to_file("wordcloud.png")

def draw_bar_graph(word_list, barcount):
    font_location = 'c:/Windows/fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=font_location).get_name()
    matplotlib.rc('font', family=font_name)

    plt.xlabel('주요 단어')
    plt.ylabel('빈도 수')
    plt.grid(True)
    #barcount = 10  # 10개만 그리겠다.

    Sorted_Dict_Values = sorted(word_list.values(), reverse=True)
    #print(Sorted_Dict_Values)
    plt.bar(range(barcount), Sorted_Dict_Values[0:barcount], align='center')

    Sorted_Dict_Keys = sorted(word_list, key=word_list.get, reverse=True)
    #print(Sorted_Dict_Keys)
    plt.xticks(range(barcount), list(Sorted_Dict_Keys)[0:barcount], rotation='70')

    plt.show()

# CountVectorizer
* countvectorizer가 하는 일

    1) 각 문서를 토큰 리스트로 변환
    
    2) 각 문서에서 토큰의 출현 빈도를 센다

    3) 각 문서를 BOW 인코딩 벡터로 변환

In [16]:
# bow

def get_count_vector(sentence_list):
    # 1글자도 인식이 되도록 토큰 패턴 변경
    cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
    cv.fit(sentence_list)
    #print(cv.vocabulary_)
    cv_array = cv.transform(sentence_list).toarray()
    return cv_array

sentence_list = ['나는 꿈을 꾸고 있다', '그런데 그 꿈은 언제쯤 이뤄질까', '조금만 꿈을 위해더 힘내보자']

get_count_vector(sentence_list)

array([[0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1]], dtype=int64)

In [21]:
vect = CountVectorizer(stop_words=["그런데", "언제쯤"]).fit(sentence_list)
vect.vocabulary_

{'나는': 3,
 '꿈을': 2,
 '꾸고': 0,
 '있다': 6,
 '꿈은': 1,
 '이뤄질까': 5,
 '조금만': 7,
 '위해더': 4,
 '힘내보자': 8}

In [24]:
import nltk

vect = CountVectorizer(tokenizer=nltk.word_tokenize).fit(sentence_list)
vect.vocabulary_

{'나는': 5,
 '꿈을': 4,
 '꾸고': 2,
 '있다': 9,
 '그런데': 1,
 '그': 0,
 '꿈은': 3,
 '언제쯤': 6,
 '이뤄질까': 8,
 '조금만': 10,
 '위해더': 7,
 '힘내보자': 11}

# TF-IDF

https://wikidocs.net/21698

In [37]:
# tfidf
def get_tfidf_vector(sentence_list):
    #galexy_tfidv = TfidfVectorizer(stop_words=["스마트폰"]).fit(galexy_top_nouns)
    tfidv = TfidfVectorizer()
    tfidv.fit(sentence_list)
    # print(tfidv.vocabulary_)
    tfidv_array = tfidv.transform(sentence_list).toarray()
    return tfidv_array


In [ ]:
def get_smiliarity_by_count_vector(text1, text2):
    cv1 = get_count_vector(text1)
    cv2 = get_count_vector(text2)
    cosine_sim = cosine_similarity(cv1, cv2)
    return cosine_sim

In [ ]:
def get_smiliarity_by_tfidf_vector(text1, text2):
    tv1 = get_tfidf_vector(text1)
    tv2 = get_tfidf_vector(text2)
    cosine_sim = linear_kernel(tv1, tv2)
    return cosine_sim

In [ ]:
def get_most_similar_items(item, series, num):
    return

In [ ]:
def create_word2vec_model(words):
    data = word2vec.LineSentence(words)
    model = word2vec.Word2Vec(data, size=200, window=10, hs=1, min_count=2, sg=1)
    model_name = 'word2vec.model'
    model.save(model_name)
    print('파일 ', model_name, '저장 완료')

In [ ]:
def draw_word2vec_bar_graph(words, x_start, x_end):
    font_location = 'c:/Windows/fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=font_location).get_name()
    matplotlib.rc('font', family=font_name)
    su = len(words)  # 전체 데이터 수
    # 축에 보여질 항목 이름들
    item = list(item[0] for item in words)
    # 그려지는 수치 데이터
    count = list(item[1] for item in words)
    plt.barh(range(su), count, align='center')
    plt.yticks(range(su), item, rotation='10')
    plt.xlim(x_start, x_end)
    plt.grid(True)
    plt.show()

In [ ]:
def get_most_similar_item_by_word2vec(model, item_list) :
    #model.wv.most_similar(positive=["한국", "트럼프"], negative=["미국"]))
    return model.wv.most_similar(positive=item_list)

In [ ]:
def get_email_address(text) :
    return re.search(r'[\w.-]+@[\w.-]+', text).group()

In [ ]:
def print_word_similarity_by_word2vec(word_list, embed) :
    #word_list = ["coffee", "cafe", "football", "soccer"]
    embeddings = embed(word_list)
    for i in range(len(word_list)):
        for j in range(i, len(word_list)):
            print("(", word_list[i], ",", word_list[j], ")", np.inner(embeddings[i], embeddings[j]))

In [ ]:
def load_word2vec_wiki250_model() :
    # pip install tensorflow_hub
    # gpu 사용하므로 colab에서 사용 권장
    # import tensorflow_hub as hub
    # hub.load("https://tfhub.dev/google/Wiki-words-250-with-normalization/2")
    return

In [ ]:
def get_review_text(text):
    return re.sub("[^a-zA-Z]", text).group()

In [ ]:
def get_most_similar_items(index, series, cos_sim, num):

    scores = list(enumerate(cos_sim[index])) # 이뉴머레이트는 각각의 값에다가 index를 달아주는 것을 말한다.   => 이걸 list로 만들어준다는 코드이다.
    scores = sorted(scores, key=lambda x:x[1], reverse=True)
    scores = scores[1:num]
    indices = [x[0] for x in scores]
    ############이 코드 못 끝냄 ㅠㅠ 보러가다!!!